# First Quantization

SELECT and PREPARE for the first quantized chemistry Hamiltonian.

In [ ]:
from qualtran import Bloq, CompositeBloq, BloqBuilder, Signature, Register
from qualtran.drawing import show_bloq
from typing import *
import numpy as np

## `PrepareTFirstQuantization`
PREPARE for the kinetic energy operator for the first quantized chemistry Hamiltonian.

This prepares the state

$$
    |+\rangle\sum_{j=1}^{\eta}|j\rangle\sum_{w=0}^{2}|w\rangle
    \sum_{r=0}^{n_{p}-2}2^{r/2}|r\rangle
    \sum_{s=0}^{n_{p}-2}2^{s/2}|s\rangle
$$

We assume that the uniform superposition over $j$ has already occured via
UniformSuperPositionIJFirstQuantization.

#### Parameters
 - `num_bits_p`: The number of bits to represent each dimension of the momentum register.
 - `eta`: The number of electrons.
 - `num_bits_rot_aa`: The number of bits of precision for the single qubit rotation for amplitude amplification. Called $b_r$ in the reference. 

#### Registers
 - `plus`: A $|+\rangle$ state register.
 - `w`: a register to index one of three components of the momenta.
 - `r`: a register encoding bits for each component of the momenta.
 - `s`: a register encoding bits for each component of the momenta. 

#### References
[Fault-Tolerant Quantum Simulations of Chemistry in First Quantization]( https://arxiv.org/abs/2105.12767) page 19, section B


In [ ]:
from qualtran.bloqs.chemistry.first_quantization import PrepareTFirstQuantization

num_bits_p = 6
eta = 10
bloq = PrepareTFirstQuantization(num_bits_p, eta)
show_bloq(bloq)

## `SelectTFirstQuantization`
SELECT for the kinetic energy operator for the first quantized chemistry Hamiltonian.

#### Parameters
 - `num_bits_p`: The number of bits to represent each dimension of the momentum register.
 - `eta`: The number of electrons. 

#### Registers
 - `sys`: The system register.
 - `plus`: A $|+\rangle$ state.
 - `flag_T`: a flag to control on the success of the $T$ state preparation. 

#### References
[Fault-Tolerant Quantum Simulations of Chemistry in First Quantization]( https://arxiv.org/abs/2105.12767) page 20, section B


In [ ]:
from qualtran.bloqs.chemistry.first_quantization import SelectTFirstQuantization

num_bits_p = 6
eta = 10
bloq = SelectTFirstQuantization(num_bits_p, eta)
show_bloq(bloq)

## `SelectUVFirstQuantization`
SELECT for the U and V operators for the first quantized chemistry Hamiltonian.

This does not include the controlled swaps from p_i and q_j into ancilla registers and back again.

#### Parameters
 - `num_bits_p`: The number of bits to represent each dimension of the momentum register.
 - `eta`: The number of electrons.
 - `num_bits_nuc_pos`: The number of bits to store each component of the nuclear positions. $n_R$ in the reference. 

Registers:

#### References
[Fault-Tolerant Quantum Simulations of Chemistry in First Quantization]( https://arxiv.org/abs/2105.12767)


In [ ]:
from qualtran.bloqs.chemistry.first_quantization import SelectUVFirstQuantization

num_bits_p = 6
eta = 10
bloq = SelectUVFirstQuantization(num_bits_p, eta, 10)
show_bloq(bloq)

## `PrepareUVFistQuantization`
PREPARE the U and V parts of the Hamiltonian.

#### Parameters
 - `num_bits_p`: The number of bits to represent each dimension of the momentum register.
 - `eta`: The number of electrons.
 - `num_atoms`: The number of atoms. $L$ in the reference.
 - `m_param`: $\mathcal{M}$ in the reference.
 - `lambda_zeta`: sum of nuclear charges.
 - `num_bits_nuc_pos`: The number of bits of precision for representing the nuclear coordinates. 

#### Registers
 - `mu`: The state controlling the nested boxes procedure.
 - `nu`: The momentum transfer register.
 - `m`: an ancilla register in a uniform superposition.
 - `l`: The register for atomic species.
 - `flag_nu`: Flag for success of the state preparation. 

#### References
[Fault-Tolerant Quantum Simulations of Chemistry in First Quantization]( https://arxiv.org/abs/2105.12767) page 19, section B


In [ ]:
from qualtran.bloqs.chemistry.first_quantization import PrepareUVFistQuantization

num_bits_p = 6
num_atoms = 10
eta = 10
m_param = 2**8
lambda_zeta = 10
num_bits_nuc_pos = 4
prep = PrepareUVFistQuantization(
    num_bits_p, eta, num_atoms, m_param, lambda_zeta, num_bits_nuc_pos
)
bloq = prep
show_bloq(bloq)

## `PrepareNuState`
PREPARE for the $\nu$ state for the $U$ and $V$ components of the Hamiltonian.

Prepares a state of the form

$$
    \frac{1}{\sqrt{\mathcal{M}2^{n_p + 2}}}
    \sum_{\mu=2}^{n_p+1}\sum_{\nu \in B_\mu}
    \sum_{m=0}^{\lceil \mathcal M(2^{\mu-2}/\lVert\nu\rVert)^2\rceil-1}
    \frac{1}{2^\mu}|\mu\rangle|\nu_x\rangle|\nu_y\rangle|\nu_z\rangle|m\rangle|0\rangle
$$

#### Parameters
 - `num_bits_p`: The number of bits to represent each dimension of the momentum register.
 - `m_param`: $\mathcal{M}$ in the reference.
 - `lambda_zeta`: sum of nuclear charges.
 - `er_lambda_zeta`: eq 91 of the referce. Cost of erasing qrom. 

#### Registers
 - `mu`: The state controlling the nested boxes procedure.
 - `nu`: The momentum transfer register.
 - `m`: an ancilla register in a uniform superposition.
 - `flag_nu`: Flag for success of the state preparation. 

#### References
[Fault-Tolerant Quantum Simulations of Chemistry in First Quantization]( https://arxiv.org/abs/2105.12767) page 19, section B


In [ ]:
from qualtran.bloqs.chemistry.first_quantization import PrepareNuState

num_bits_p = 6
m_param = 2 ** (2 * num_bits_p + 3)
bloq = PrepareNuState(num_bits_p, m_param)
show_bloq(bloq)

In [ ]:
from qualtran.resource_counting import get_bloq_counts_graph
from qualtran.drawing import GraphvizCounts
graph, sigma = get_bloq_counts_graph(bloq)
GraphvizCounts(graph).get_svg()